In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Proj 
import os
import shutil
import PIL
import xarray as xr
import geopandas as gp
from functools import partial
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import glob
import dask
from shapely.geometry import MultiPoint
from shapely.geometry import Point
from shapely.geometry import Polygon

In [67]:
!pip install descartes

In [4]:
inputdir = '/home/mor582/seagrass'
country = 'TH'
cameratype = 'RGB'
dronetype='DJIP4'
surveycode='TKB'
surveynumber=5
timedif = '+07'
inputpath =os.path.join(inputdir,country,f'{surveycode}_{surveynumber:03}')

In [105]:
data = pd.read_csv(f'{inputpath}/{dronetype}_{country}_{surveycode}_{surveynumber:03}_IMG_GP.CSV',index_col='Time',parse_dates=['Time'])
data.DateTimeOriginal  = pd.to_datetime(data.DateTimeOriginal,format='%Y-%m-%d %H:%M:%S')
crs = data.iloc[0].Crs

area = gp.read_file(f'{inputpath}/{country}_{surveycode}_surveyarea.shp').to_crs(crs=crs)

In [114]:
from shapely import wkt

data['geometry'] = data['ImagePolygon'].apply(wkt.loads)

gdf = gp.GeoDataFrame(
    data,
    crs=crs)

gdf['OrigArea'] = gdf.geometry.area

In [119]:
intersection = gp.overlay(
    gdf,
    area,
    how="intersection"
)

intersection['SurveyPercent']=(intersection.geometry.area /intersection.OrigArea)*100
intersection = intersection[['FileName','SurveyPercent']]
gdf =gdf.merge(intersection,on='FileName',how='left')

In [120]:
gdf.columns

Index(['SourceFile', 'ExifToolVersion', 'FileName', 'Directory', 'FileSize',
       'FileModifyDate', 'FileAccessDate', 'FileInodeChangeDate',
       'FilePermissions', 'FileType',
       ...
       'ImageNorthing', 'ImageEasting', 'ImageDistance', 'ImageSpeed',
       'geometry', 'ImagePolygon', 'InArea', 'OrigArea', 'SuerveyPercent',
       'SurveyPercent'],
      dtype='object', length=160)

In [121]:
gdf.SurveyPercent

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
5             NaN
6             NaN
7             NaN
8             NaN
9             NaN
10            NaN
11            NaN
12            NaN
13            NaN
14            NaN
15            NaN
16            NaN
17            NaN
18            NaN
19            NaN
20            NaN
21            NaN
22            NaN
23            NaN
24            NaN
25            NaN
26            NaN
27            NaN
28            NaN
29            NaN
30            NaN
31            NaN
32            NaN
33            NaN
34            NaN
35            NaN
36            NaN
37            NaN
38            NaN
39            NaN
40            NaN
41            NaN
42            NaN
43            NaN
44            NaN
45            NaN
46            NaN
47            NaN
48            NaN
49            NaN
50            NaN
51            NaN
52            NaN
53            NaN
54            NaN
55        

In [122]:
selection =gdf[gdf.SurveyPercent>90].sample(30)
outputpath =inputpath+'/Segemented'

In [123]:
os.makedirs(outputpath,exist_ok=True)

In [124]:
from PIL import Image 

In [129]:
selection.iloc[20].FileName

'DJIP4_TH_TKB_005_20181222T173030+07_0259.JPG'

In [32]:
!pip install image_slicer

In [130]:
from image_slicer import slice
for index,row in selection.iterrows():
    tiles =slice(os.path.join(inputpath,row.FileName), 4,save=False)
    for tile in tiles:
        tile.save(filename =tile.generate_filename(outputpath,os.path.splitext(row.FileName)[0]))
    